In [1]:
import os
import numpy as np
import pandas as pd
from random import randint,seed
from tqdm import tqdm
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,MinMaxScaler 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

In [2]:
df1 = pd.read_csv('data_conv/max_conv1.csv')
df1['freq'] = 1200
print (df1.shape)
df2 = pd.read_csv('data_conv/max_conv2.csv')
df2['freq'] = 800
print (df2.shape)
df3 = pd.read_csv('data_conv/max_conv3.csv')
df3['freq'] = 600
print (df3.shape)
df4 = pd.read_csv('data_conv/max_conv4.csv')
df4['freq'] = 1000
print (df4.shape)

(2500, 8)
(2500, 8)
(2500, 8)
(2500, 8)


In [3]:
df = pd.concat([df1, df2,df3,df4], ignore_index=True)

In [4]:
df.head()

,Unnamed: 0,input_size_h,input_size_w,channels,filter_size,stride,runtime,freq
0,0,89,123,58,3,2,0.032802,1200
1,1,214,295,6,2,2,0.011058,1200
2,2,266,35,101,3,2,0.047136,1200
3,3,37,142,123,2,3,0.008245,1200
4,4,47,19,131,2,3,0.001566,1200


In [5]:
df = df.drop(columns=['Unnamed: 0'])
df.head()

,input_size_h,input_size_w,channels,filter_size,stride,runtime,freq
0,89,123,58,3,2,0.032802,1200
1,214,295,6,2,2,0.011058,1200
2,266,35,101,3,2,0.047136,1200
3,37,142,123,2,3,0.008245,1200
4,47,19,131,2,3,0.001566,1200


In [6]:
df_ = df.dropna(axis=0)
df_.head()

,input_size_h,input_size_w,channels,filter_size,stride,runtime,freq
0,89,123,58,3,2,0.032802,1200
1,214,295,6,2,2,0.011058,1200
2,266,35,101,3,2,0.047136,1200
3,37,142,123,2,3,0.008245,1200
4,47,19,131,2,3,0.001566,1200


In [7]:
df_.shape

(10000, 7)

In [8]:
df_['in_size'] = df_.input_size_h*df_.input_size_w

In [9]:
df_.head()

,input_size_h,input_size_w,channels,filter_size,stride,runtime,freq,in_size
0,89,123,58,3,2,0.032802,1200,10947
1,214,295,6,2,2,0.011058,1200,63130
2,266,35,101,3,2,0.047136,1200,9310
3,37,142,123,2,3,0.008245,1200,5254
4,47,19,131,2,3,0.001566,1200,893


In [10]:
data = df_[['freq','in_size' ,'channels','filter_size' ,'stride']]
data = data
data.head()

,freq,in_size,channels,filter_size,stride
0,1200,10947,58,3,2
1,1200,63130,6,2,2
2,1200,9310,101,3,2
3,1200,5254,123,2,3
4,1200,893,131,2,3


In [11]:
data.max(axis=0)

freq             1200
in_size        200256
channels        66785
filter_size         3
stride              3
dtype: int64

In [12]:
data.describe()

,freq,in_size,channels,filter_size,stride
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,900.000000,50812.466300,111.595200,2.497200,2.484200
std,223.617979,44565.103857,1236.778045,0.500017,0.499775
min,600.000000,8.000000,1.000000,2.000000,2.000000
25%,750.000000,14159.250000,5.000000,2.000000,2.000000
50%,900.000000,38476.000000,12.000000,2.000000,2.000000
75%,1050.000000,77441.250000,33.000000,3.000000,3.000000
max,1200.000000,200256.000000,66785.000000,3.000000,3.000000


In [13]:
target = np.log10(df_.runtime*1000)
target.head()

0    1.515900
1    1.043677
2    1.673353
3    0.916191
4    0.194792
Name: runtime, dtype: float64

In [14]:
scaler = StandardScaler() 
#scaler = MinMaxScaler()
scaler.fit(data)  
data = scaler.transform(data)  
#X_test = scaler.transform(X_test) 
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=0)

In [15]:
new_df = pd.DataFrame(X_train)
new_df.describe()

,0,1,2,3,4
count,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,0.005814,0.008029,0.001720,0.000850,-0.001401
std,1.001644,1.005193,1.027832,1.000067,1.000017
min,-1.341641,-1.140063,-0.089426,-0.994416,-0.968884
25%,-0.447214,-0.820301,-0.086192,-0.994416,-0.968884
50%,0.447214,-0.262796,-0.080532,-0.994416,-0.968884
75%,1.341641,0.607233,-0.063552,1.005616,1.032115
max,1.341641,3.353543,53.911645,1.005616,1.032115


In [16]:
model = LinearRegression()
model.fit(X_train, y_train)
print('accuracy: ',model.score(X_test, y_test)*100)
print (model.coef_)
pred = model.predict(X_test)
print 'nonzero: ',pred.shape[0]-pred[pred<0].shape[0],'/',pred.shape[0]

('accuracy: ', 25.482500951322884)
[-0.10087407  0.03509439  0.01837085  0.11771525 -0.16633186]
nonzero:  2000 / 2000


In [17]:
model = linear_model.Ridge(alpha=.5)
model.fit(X_train, y_train) 
print('accuracy: ',model.score(X_test, y_test)*100)
print (model.coef_)
pred = model.predict(X_test)
print 'nonzero: ',pred.shape[0]-pred[pred<0].shape[0],'/',pred.shape[0]

('accuracy: ', 25.48268541948049)
[-0.1008678   0.03509205  0.01836947  0.11770803 -0.16632141]
nonzero:  2000 / 2000


In [18]:
model = linear_model.Lasso(alpha=0.1)
model.fit(X_train, y_train)
print('accuracy: ',model.score(X_test, y_test)*100)
print (model.coef_)
pred = model.predict(X_test)
print 'nonzero: ',pred.shape[0]-pred[pred<0].shape[0],'/',pred.shape[0]

('accuracy: ', 11.248955080834632)
[-0.00172574  0.          0.          0.01965243 -0.06556741]
nonzero:  2000 / 2000


In [19]:
model = SVR(kernel='rbf', C=1e3, gamma=0.1)
model.fit(X_train, y_train)
print('accuracy: ',model.score(X_test, y_test)*100)
pred = model.predict(X_test)
print 'nonzero: ',pred.shape[0]-pred[pred<0].shape[0],'/',pred.shape[0]

('accuracy: ', 78.52657027510725)
nonzero:  1997 / 2000


In [20]:
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
model = MLPRegressor(hidden_layer_sizes=1024,learning_rate='invscaling',early_stopping=True,max_iter=1000)

In [21]:
scaler.fit(data)  
data_ = scaler.transform(data)
scores = cross_val_score(model, data_, target, cv=cv) 
scores

array([0.27563588, 0.72408356, 0.30289745, 0.73849912, 0.78653047])

In [22]:
model = MLPRegressor(hidden_layer_sizes=2048,learning_rate='invscaling',early_stopping=True,max_iter=10000)
model.fit(X_train, y_train)
print('accuracy: ',model.score(X_test, y_test)*100)
pred = model.predict(X_test)
print 'nonzero: ',pred.shape[0]-pred[pred<0].shape[0],'/',pred.shape[0]

('accuracy: ', 81.36248598789453)
nonzero:  1990 / 2000


In [23]:
bias = np.asarray(model.intercepts_)
arr = np.asarray(model.coefs_)

In [33]:
if os.path.exists("params_2.txt"):
    os.remove("params_2.txt")
f=open('params_2.txt','ab')
print >> f, "L0"+" W"+" "+str(arr[0].shape[0])+" "+str(arr[0].shape[1]) 
np.savetxt(f,arr[0])
print >> f, "L0"+" B"+" "+str(bias[0].shape[0])+" 1"
np.savetxt(f,bias[0])

print >> f, "L1"+" W"+" "+str(arr[1].shape[0])+" "+str(arr[1].shape[1]) 
np.savetxt(f,arr[1])
print >> f, "L1"+" B"+" "+str(bias[1].shape[0])+" 1"
np.savetxt(f,bias[1])
f.close()

In [34]:
print X_test.shape
np.savetxt('input_2.txt',X_test)

print model.predict(X_test).shape
np.savetxt('output_2.txt',model.predict(X_test))


(2000, 5)
(2000,)


In [32]:
model = KNeighborsRegressor(n_neighbors=10, weights='distance')
model.fit(X_train, y_train)
print('accuracy: ',model.score(X_test, y_test)*100)
pred = model.predict(X_test)
print 'nonzero: ',pred.shape[0]-pred[pred<0].shape[0],'/',pred.shape[0]

('accuracy: ', 79.16437324660866)
nonzero:  1990 / 1990
